In [1]:
import pandas as pd
import numpy as np
import re
import requests
import time

## Seleccionamos el numero de jornada

In [2]:
jornada = 12

In [3]:
url= "https://fbref.com/es/comps/12/horario/Resultados-y-partidos-en-La-Liga"
tables = pd.read_html(url)
df = tables[0]
# seleccionamos las variables
df = df[['Sem.', 'Día', 'Fecha','Local','Visitante']]
## Hacemos la columna fecha del formato correspondiente
df["Fecha"] = pd.to_datetime(df["Fecha"])
# Hacemos el encoding de los días
dias_map = {
    'Lun': 1,
    'Mar': 2,
    'Mié': 3,
    'Jue': 4,
    'Vie': 5,
    'Sáb': 6,
    'Dom': 7
}
df["Día"] = df["Día"].map(dias_map)
# Filtramos por jornada
df = df[df["Sem."] == jornada]
# Agregamos la columnda de sede
df["Sedes"] = 1
# Renombramos las columnas
df = df[["Día","Sedes","Visitante","Local"]]
df = df.rename(columns = {"Local":"Anfitrion","Visitante":"Adversario"})
# Duplicamos el dataframe e invertimos las columnas para hacer la concatenacion
df_2 = df.copy()
df_2 = df_2.rename(columns={"Adversario":"Anfitrion","Anfitrion":"Adversario"})
df_2["Sedes"] = 0
df = pd.concat([df, df_2], ignore_index=True)
df["Día"] = df["Día"].astype(int)

C:\Users\ctorr\AppData\Local\Temp\ipykernel_12432\3972242974.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Fecha"] = pd.to_datetime(df["Fecha"])
C:\Users\ctorr\AppData\Local\Temp\ipykernel_12432\3972242974.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Día"] = df["Día"].map(dias_map)


In [4]:
df

,Día,Sedes,Adversario,Anfitrion
0,5,1,Mallorca,Alavés
1,6,1,Valladolid,Osasuna
2,6,1,Leganés,Girona
3,6,1,Rayo Vallecano,Villarreal
4,6,1,Real Madrid,Valencia
5,7,1,Las Palmas,Atlético Madrid
6,7,1,Espanyol,Barcelona
7,7,1,Real Sociedad,Sevilla
8,7,1,Betis,Athletic Club
9,1,1,Getafe,Celta Vigo


### Estadisticas básicas

In [5]:
url= "https://fbref.com/es/comps/12/Estadisticas-de-La-Liga"
tables = pd.read_html(url)
df_basic = tables[0]

In [6]:
df_basic=df_basic[['RL','Equipo','PG','PE', 'PP','GF','GC', 'xG', 'xGA','Últimos 5','Máximo Goleador del Equipo']]

In [7]:
df_basic['Máximo Goleador del Equipo'] = df_basic['Máximo Goleador del Equipo'].apply(lambda x: int(re.search(r'\b(\d+)\b', x).group(1)) if re.search(r'\b(\d+)\b', x) else None)

df_basic['Últimos 5'] = df_basic['Últimos 5'].apply(lambda resultados: sum([3 if resultado == 'PG' else (1 if resultado == 'PE' else 0) for resultado in resultados.split()]))


C:\Users\ctorr\AppData\Local\Temp\ipykernel_12432\339425139.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_basic['Máximo Goleador del Equipo'] = df_basic['Máximo Goleador del Equipo'].apply(lambda x: int(re.search(r'\b(\d+)\b', x).group(1)) if re.search(r'\b(\d+)\b', x) else None)
C:\Users\ctorr\AppData\Local\Temp\ipykernel_12432\339425139.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_basic['Últimos 5'] = df_basic['Últimos 5'].apply(lambda resultados: sum([3 if resultado == 'PG' else (1 if 

### Estadisticas de Ofensiva

In [8]:
url= "https://fbref.com/es/comps/12/Estadisticas-de-La-Liga"
tables = pd.read_html(url)
df_ataque = tables[2]

In [9]:
df_ataque = df_ataque.drop(["Tiempo Jugado","Expectativa",'Por 90 Minutos'],axis=1)

C:\Users\ctorr\AppData\Local\Temp\ipykernel_12432\1442764144.py:1: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  df_ataque = df_ataque.drop(["Tiempo Jugado","Expectativa",'Por 90 Minutos'],axis=1)


In [10]:
df_ataque.columns = df_ataque.columns.droplevel(level=0)

In [11]:
df_ataque=df_ataque[['Equipo', 'Edad', 'Pos.','Ass','TPint', 'PrgC', 'PrgP']]

##### Disparos

In [12]:
url= "https://fbref.com/es/comps/12/Estadisticas-de-La-Liga"
tables = pd.read_html(url)
df_disparos = tables[8]

In [13]:
df_disparos.columns=df_disparos.columns.droplevel(level=0)

In [14]:
df_disparos=df_disparos[['Equipo', '% de TT','Dist']]

In [15]:
df_ataque = pd.merge(df_ataque, df_disparos, left_on='Equipo', right_on='Equipo', how='left')

##### Pases

In [16]:
url= "https://fbref.com/es/comps/12/Estadisticas-de-La-Liga"
tables = pd.read_html(url)
df_pases = tables[10]

In [17]:
df_pases = df_pases.drop(["Cortos","Medios",'Largos','Expectativa'],axis=1)

C:\Users\ctorr\AppData\Local\Temp\ipykernel_12432\466504704.py:1: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  df_pases = df_pases.drop(["Cortos","Medios",'Largos','Expectativa'],axis=1)


In [18]:
df_pases.columns=df_pases.columns.droplevel(level=0)

In [19]:
df_pases=df_pases[['Equipo', '% Cmp','Dist. tot.']]

In [20]:
df_ataque = pd.merge(df_ataque, df_pases, left_on='Equipo', right_on='Equipo', how='left')

### Estadisticas de defensa

In [21]:
url= "https://fbref.com/es/comps/12/Estadisticas-de-La-Liga"
tables = pd.read_html(url)
df_porteria = tables[4]

In [22]:
df_porteria = df_porteria.drop(["Tiempo Jugado","Tiros penales"],axis=1)

C:\Users\ctorr\AppData\Local\Temp\ipykernel_12432\2417922220.py:1: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  df_porteria = df_porteria.drop(["Tiempo Jugado","Tiros penales"],axis=1)


In [23]:
df_porteria.columns = df_porteria.columns.droplevel(level=0)

In [24]:
df_porteria=df_porteria[['Equipo', 'GC', 'DaPC', 'Salvadas','PaC']]

In [25]:
url= "https://fbref.com/es/comps/12/Estadisticas-de-La-Liga"
tables = pd.read_html(url)
df_defensa = tables[16]

In [26]:
df_defensa = df_defensa.drop(['Desafíos'],axis=1)

C:\Users\ctorr\AppData\Local\Temp\ipykernel_12432\1078235159.py:1: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  df_defensa = df_defensa.drop(['Desafíos'],axis=1)


In [27]:
df_defensa.columns=df_defensa.columns.droplevel(level=0)

In [28]:
df_defensa=df_defensa[['Equipo', 'TklG','Int','Err']]

In [29]:
df_final = pd.merge(df_ataque, df_defensa, left_on='Equipo', right_on='Equipo', how='left')

In [30]:
df_final = pd.merge(df_final, df_basic, left_on='Equipo', right_on='Equipo', how='left')

In [31]:
df_opp=df_final.copy()
df_tm=df_final.copy()

In [32]:
columns_to_rename = ['Edad', 'Pos.', 'Ass', 'TPint', 'PrgC', 'PrgP', '% de TT',
       'Dist', '% Cmp', 'Dist. tot.', 'TklG', 'Int', 'Err', 'RL', 'PG', 'PE',
       'PP', 'GF', 'GC', 'xG', 'xGA', 'Últimos 5',
       'Máximo Goleador del Equipo']
new_column_names_tm = [f"{col}(tm)" for col in columns_to_rename]
df_tm.rename(columns=dict(zip(columns_to_rename, new_column_names_tm)), inplace=True)

In [33]:
columns_to_rename = ['Edad', 'Pos.', 'Ass', 'TPint', 'PrgC', 'PrgP', '% de TT',
       'Dist', '% Cmp', 'Dist. tot.', 'TklG', 'Int', 'Err', 'RL', 'PG', 'PE',
       'PP', 'GF', 'GC', 'xG', 'xGA', 'Últimos 5',
       'Máximo Goleador del Equipo']
new_column_names_opp = [f"{col}(opp)" for col in columns_to_rename]
df_opp.rename(columns=dict(zip(columns_to_rename, new_column_names_opp)), inplace=True)

In [34]:
df = pd.merge(df, df_opp, left_on='Adversario', right_on='Equipo', how='left')

In [35]:
df = pd.merge(df, df_tm, left_on='Anfitrion', right_on='Equipo', how='left')

In [36]:
df=df.drop(['Equipo_x','Equipo_y'],axis=1)

In [37]:
df.head()

,Día,Sedes,Adversario,Anfitrion,Edad(opp),Pos.(opp),Ass(opp),TPint(opp),PrgC(opp),PrgP(opp),...,RL(tm),PG(tm),PE(tm),PP(tm),GF(tm),GC(tm),xG(tm),xGA(tm),Últimos 5(tm),Máximo Goleador del Equipo(tm)
0,5,1,Mallorca,Alavés,28.9,45.8,7,1,143,345,...,16,3,1,7,13,19,11.9,16.5,0,3
1,6,1,Valladolid,Osasuna,25.8,43.0,6,3,143,317,...,8,5,3,3,16,16,11.5,14.2,8,5
2,6,1,Leganés,Girona,28.0,43.6,9,1,120,290,...,13,3,3,5,11,14,13.9,12.5,5,3
3,6,1,Rayo Vallecano,Villarreal,29.4,48.8,9,0,161,372,...,3,6,3,2,20,19,17.9,14.8,10,7
4,6,1,Real Madrid,Valencia,27.7,58.1,15,5,260,581,...,20,1,4,6,8,17,8.0,17.9,3,2


### Exportamos el archivo

In [38]:
archivo_destino = f"Predictions Jornada{jornada}.xlsx"

# Escribe el dataframe en el archivo de destino
df.to_excel(archivo_destino)

print(f"Se agregron correctamente los datos de {archivo_destino}")

Se agregron correctamente los datos de Predictions Jornada12.xlsx
